<a href="https://colab.research.google.com/github/itsabbhii/Training-AIML/blob/main/NLP/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!git clone https://github.com/itsabbhii/Training-AIML.git
%cd Training-AIML/NLP/

Cloning into 'Training-AIML'...
remote: Enumerating objects: 1124, done.
remote: Counting objects: 100% (922/922), done.
remote: Compressing objects: 100% (904/904), done.
remote: Total 1124 (delta 48), reused 858 (delta 14), pack-reused 202 (from 1)
Receiving objects: 100% (1124/1124), 54.22 MiB | 11.67 MiB/s, done.
Resolving deltas: 100% (122/122), done.
/content/Training-AIML/NLP/Training-AIML/NLP


In [18]:
import nltk,string,pickle
import random
from nltk.corpus import movie_reviews,stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,confusion_matrix

In [19]:
nltk.download(['movie_reviews', 'stopwords', 'wordnet'])
documents = [
    (list(movie_reviews.words(fileid)), category)
    for category in movie_reviews.categories()
    for fileid in movie_reviews.fileids(category)
]
random.shuffle(documents)
reviews, sentiments = zip(*documents)

[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [21]:
def preprocess(review):
  lemmatizer = WordNetLemmatizer()
  stopword = set(stopwords.words('english'))
  return ''.join([lemmatizer.lemmatize(word.lower()) for word in review if word not in string.punctuation and word.lower() not in stopword])
processed_reviews = [preprocess(review) for review in reviews]

In [22]:
X_train, X_test, y_train, y_test = train_test_split(processed_reviews, sentiments, test_size=0.2, random_state=42)